# Different Technologies

### AUTENTIFICATION E CREATE CHAT

In [1]:
import requests


BASE_URL = "http://localhost:3000"
url = f"{BASE_URL}/api/v1/auths/signin"
payload = {
    "email": "eleonora.fabris@dbagroup.it", 
    "password": "Scoiattolo1!"
}
response = requests.post(url, json=payload)
token = response.json()["token"]

url = f"{BASE_URL}/api/v1/models"
JWT_TOKEN = token

HEADERS_JSON = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}
HEADERS_FILE = {
    "Authorization": f"Bearer {JWT_TOKEN}"
}

params = {
        "max_new_tokens": 50, 
        "temperature": 0.7, # se 0.0 risponde sempre allo stesso modo, estremamente preciso
        "top_p": 0.9, # indica la variabilità del vocabolario
    }

response = requests.get(url, headers=HEADERS_JSON)

if response.status_code == 200:
    print("Autenticazione riuscita!")
    print(response.json()) # lista modelli disponibili
else:
    raise Exception(f"Errore nell'autenticazione: {response.status_code}")

Autenticazione riuscita!
{'data': [{'id': 'mistral:latest', 'name': 'mistral:latest', 'object': 'model', 'created': 1768897279, 'owned_by': 'ollama', 'ollama': {'name': 'mistral:latest', 'model': 'mistral:latest', 'modified_at': '2026-01-14T11:14:48.1442843+01:00', 'size': 4372824384, 'digest': '6577803aa9a036369e481d648a2baebb381ebc6e897f2bb9a766a2aa7bfbc1cf', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '7.2B', 'quantization_level': 'Q4_K_M'}, 'connection_type': 'local', 'urls': [0]}, 'connection_type': 'local', 'tags': [], 'actions': [], 'filters': []}, {'id': 'qwen3-vl:latest', 'name': 'qwen3-vl:latest', 'object': 'model', 'created': 1768897279, 'owned_by': 'ollama', 'ollama': {'name': 'qwen3-vl:latest', 'model': 'qwen3-vl:latest', 'modified_at': '2026-01-14T11:14:48.9312939+01:00', 'size': 6140415879, 'digest': '901cae73216286ea8c5aba8b46d307ff7188f737285ec500c795a12f05225d28', 'details': {'parent_model': '', 'format

### LOAD FILE

In [2]:
import requests
import json

# pdf_name = "NED19RC.20_EN_2025A_web_20251014.pdf" # pv
# pdf_file_path = rf"C:\Users\eleonora.fabris\Downloads\Technolgies\Pannelli fotovoltaici Trina Solar\{pdf_name}"

pdf_name = "250227_Datasheet_VertexS_NEG9RC.27_EN_2024_B_web.pdf" # inverter
pdf_file_path = rf"C:\Users\eleonora.fabris\Downloads\Technolgies\Pannelli fotovoltaici Trina Solar\{pdf_name}"

with open(pdf_file_path, "rb") as f:
    files = {"file": ("Datasheet.pdf", f, "application/pdf")}
    response = requests.post(f"{BASE_URL}/api/v1/files/?process=true", files=files, headers=HEADERS_FILE)

if response.status_code not in [200, 201]:
    raise Exception(f"Errore nel caricamento del file: {response.status_code} - {response.text}")

file_data = response.json()
file_id = file_data.get("id")

with requests.get(
    f"{BASE_URL}/api/v1/files/{file_id}/process/status?stream=true",
    headers=HEADERS_FILE,
    stream=True
) as r:
    if r.status_code != 200:
        raise Exception(f"Errore status SSE (Server-Sent Events): {r.status_code} - {r.text}")
    for line in r.iter_lines(decode_unicode=True):
        if not line:
            continue
        if line.startswith("data:"):
            data = json.loads(line.replace("data:", "").strip())
            status = data.get("status")
            if status == "completed":
                print(f"Caricamento del file {file_id} completato.")
                break
            if status in ("failed", "error"):
                raise Exception("Processing fallito")

Caricamento del file cb32b793-84be-484b-bdff-54e82c619489 completato.


In [3]:
from pypdf import PdfReader  

reader = PdfReader(pdf_file_path)
pdf_text = "\n".join([page.extract_text() or "" for page in reader.pages])


In [4]:
import requests
import uuid


initial_assistant_id = str(uuid.uuid4())

payload_new_chat = {
    "chat": {
        "title": "Lettura Datasheet",
        "models": ["falcon3:7b"], 
        "params": params
    }
}
url_new_chat = f"{BASE_URL}/api/v1/chats/new"
response = requests.post(url_new_chat, json=payload_new_chat, headers=HEADERS_JSON)
if response.status_code not in [200, 201]:
    raise Exception(f"Errore nella creazione chat: {response.status_code} - {response.text}")

chat_data = response.json()
chat_id = chat_data["id"]
current_id = initial_assistant_id
print("Nuova chat creata:", chat_id)

url_get_chat = f"{BASE_URL}/api/v1/chats/{chat_id}"
chat_info = requests.get(url_get_chat, headers=HEADERS_JSON).json()

Nuova chat creata: f8d1b92a-886c-4f4b-8fbf-1e1b44f0ed35


### SEND MESSAGE

In [17]:
messages_inverter = [
    {
        "role": "user",
        "content": (
            "Analizza ESCLUSIVAMENTE il contenuto del PDF riportato di seguito. "
            "Non utilizzare conoscenze esterne.\n\n"

            "=== PDF TEXT START ===\n"
            + pdf_text +
            "\n=== PDF TEXT END ===\n\n"

            "Estrai i seguenti parametri dell'inverter, se presenti nel documento "
            "e restituisci esclusivamente un JSON valido:\n"
            "{\n"
            '  "Vac": "...",\n'
            '  "Pso": "...",\n'
            '  "Paco": "...",\n'
            '  "Pdco": "...",\n'
            '  "Vdco": "...",\n'
            '  "C0": "...",\n'
            '  "C1": "...",\n'
            '  "C2": "...",\n'
            '  "C3": "...",\n'
            '  "Pnt": "...",\n'
            '  "Vdcmax": "...",\n'
            '  "Idcmax": "...",\n'
            '  "Mppt_low": "...",\n'
            '  "Mppt_high": "...",\n'
            '  "CEC_Date": "...",\n'
            '  "CEC_Type": "...",\n'
            '  "surface_type": "..." \n'
            "}\n\n"

            "Here are the definitions of each field:\n"
            "-- Vac: Nominal AC output voltage (V)\n"
            "-- Pso: DC power at which inverter starts producing AC power (W)\n"
            "-- Paco: Maximum continuous AC power output (W)\n"
            "-- Pdco: DC power required to produce Paco under reference conditions (W)\n"
            "-- Vdco: Reference DC voltage at which Pdco and Paco are defined (V)\n"
            "-- C0, C1, C2, C3: Empirical coefficients describing efficiency curve\n"
            "-- Pnt: Night tare loss (standby consumption, W)\n"
            "-- Vdcmax: Maximum DC input voltage (V)\n"
            "-- Idcmax: Maximum DC input current (A)\n"
            "-- Mppt_low, Mppt_high: MPPT operating voltage window (V)\n"
            "-- CEC_Date: Date the inverter was listed/tested by CEC\n"
            "-- CEC_Type: Application class (Utility Interactive, Microinverter, etc.)\n"
            "-- surface_type: Surface type albedo, e.g., urban, grass, snow, asphalt, etc.\n\n"

            "Rules:\n"
            "- Se un parametro non è presente nel PDF, usa il valore: \"NOT_FOUND\".\n"
            "- Non aggiungere commenti.\n"
            "- Non aggiungere testo fuori dal JSON.\n"
            "- Il JSON deve essere sintatticamente valido."
        )
    }
]


In [67]:
messages_inverter = [
    {
        "role": "user",
        "content": (
            "Analizza ESCLUSIVAMENTE il contenuto del PDF riportato di seguito. Non utilizzare conoscenze esterne.\n"
            "=== PDF TEXT START ===\n"
            + pdf_text +
            "\n=== PDF TEXT END ===\n\n"
            "Il documento può contenere una tabella tecnica con più modelli di inverter oppure uno unico. "
            "Importante: Alcuni parametri sono comuni a tutti i modelli (celle unite), mentre altri (come la sezione 'Output') sono specifici per ogni colonna/modello.\n"
            "Estrai i parametri specificati per TUTTI i modelli identificati nelle intestazioni di colonna.\n"
            "Restituisci esclusivamente un array JSON di oggetti, dove ogni oggetto rappresenta un modello.\n"
            "Usa questa struttura:\n"
            '{ "ModelName": "...", "Vac": "...", "Pso": "...", "Paco": "...", "Pdco": "...", "Vdco": "...", "C0": "...", "C1": "...", "C2": "...", "C3": "...", "Pnt": "...", "Vdcmax": "...", "Idcmax": "...", "Mppt_low": "...", "Mppt_high": "...", "surface_type": "..." }\n'
            "Istruzioni per l'estrazione:\n\n"
            "Identificazione Modelli: Crea un oggetto separato per ogni modello trovato nelle intestazioni.\n"
            "Parametri Condivisi: Se un valore nella tabella è centrato o unico per tutti i modelli, applicalo a tutti gli oggetti nel JSON.\n"
            "Parametri Specifici: Se un valore cambia per colonna, assegna il valore corretto al rispettivo modello.\n"
            "Mappatura Definizioni:\n"
            "Vac: Prendi da 'Rated Output Voltage'.\n"
            "Paco: Prendi da 'Rated AC Active Power'.\n"
            "Vdcmax: Prendi da 'Max. Input Voltage'.\n"
            "Idcmax: Prendi da 'Max. Current per MPPT'.\n"
            "Mppt_low / Mppt_high: Estrai i limiti inferiore e superiore dal 'MPPT Operating Voltage Range'.\n"
            "Se un parametro non è presente nel testo, scrivi null.\n"
            "Restituisci solo il codice JSON."
        )
    }
]


In [ ]:
messages_pv = [
    {
        "role": "user",
        "content": (
            "Analizza ESCLUSIVAMENTE il contenuto del PDF riportato di seguito. "
            "Non utilizzare conoscenze esterne.\n\n"

            "=== PDF TEXT START ===\n"
            + pdf_text +
            "\n=== PDF TEXT END ===\n\n"

            "Estrai i seguenti parametri del modulo fotovoltaico, se presenti nel documento "
            "e restituisci esclusivamente un JSON valido:\n"
            "{\n"
            '  "Technology": "...",\n'
            '  "Bifacial": "...",\n'
            '  "STC": "...",\n'
            '  "PTC": "...",\n'
            '  "A_c": "...",\n'
            '  "Length": "...",\n'
            '  "Width": "...",\n'
            '  "N_s": "...",\n'
            '  "I_sc_ref": "...",\n'
            '  "V_oc_ref": "...",\n'
            '  "I_mp_ref": "...",\n'
            '  "V_mp_ref": "...",\n'
            '  "alpha_sc": "...",\n'
            '  "beta_oc": "...",\n'
            '  "T_NOCT": "...",\n'
            '  "a_ref": "...",\n'
            '  "I_L_ref": "...",\n'
            '  "I_o_ref": "...",\n'
            '  "R_s": "...",\n'
            '  "R_sh_ref": "...",\n'
            '  "Adjust": "...",\n'
            '  "gamma_r": "...",\n'
            '  "BIPV": "...",\n'
            "}\n\n"

            "Here are the definitions of each fields:\n"
            "-- Bifacial indicates whether the PV module is bifacial (1) or not (0)\n"
            "-- STC stands for Standard Test Conditions power, corresponding to 1000 W/m² irradiance, 25 °C cell temp, AM1.5 spectrum\n"
            "-- PTC stands for PVUSA Test Conditions power, corresponding to 1000 W/m² irradiance, 20 °C ambient, 1 m/s wind, 1.5 air mass\n"
            "-- A_c represents the aperture area of the PV module (m^2), the Length is its length (m), the Width is its width (m)\n"
            "-- N_s is the number of cells connected in series in the PV module\n"
            "-- I_sc_ref (reference circuit voltage) is the currect (A) when voltage is 0 at STC\n"
            "-- V_oc_ref (reference circuit voltage) is the voltage (V) at zero current at STC\n"
            "-- I_mp_ref is the current at maximum power point at STC (A)\n"
            "-- V_mp_ref is the Voltage at maximum power point at STC (V)\n"
            "-- alpha_sc is the Temperature coefficient of short-circuit current (A/°C)\n"
            "-- beta_oc is the Temperature coefficient of open-circuit voltage (V/°C)\n"
            "-- T_NOCT is the nominal operating cell temperature\n"
            "-- a_ref is the Modified diode ideality factor × number of cells × thermal voltage\n"
            "-- I_L_ref is the Light-generated current at reference conditions (A)\n"
            "-- I_o_ref is the Diode saturation current at reference conditions (A)\n"
            "-- R_s is the series resistance (Ω)\n"
            "-- R_sh_ref is the Shunt resistance at STC (Ω)\n"
            "-- Adjust is the Empirical adjustment factor\n"
            "-- gamma_r is the Temperature coefficient of Pmp (%/°C)\n"
            "-- BIPV indicates whether the PV module is building-integrated (Y) or not (N)\n\n"

            "This is the rule:\n"
            "- Se un parametro non è presente nel PDF, usa il valore: \"NOT_FOUND\".\n"
            "- Non aggiungere commenti.\n"
            "- Non aggiungere testo fuori dal JSON.\n"
            "- Il JSON deve essere sintatticamente valido."
        )
    }
]


In [5]:
messages_pv = [
    {
        "role": "user",
        "content": (
            "Analizza ESCLUSIVAMENTE il contenuto del PDF riportato di seguito. "
            "Non utilizzare conoscenze esterne. Ignora le pagine non pertinenti o vuote.\n\n"
            "Utilizza TUTTE le tabelle tecniche presenti nel PDF."

            "=== PDF TEXT START ===\n"
            + pdf_text +
            "\n=== PDF TEXT END ===\n\n"

            "Il documento contiene tabelle con più modelli (es. 490M, 495M, 500M, 505M). "
            "Estrai i parametri per OGNI modello identificato nelle colonne delle tabelle "
            "e restituisci esclusivamente un array JSON di oggetti (uno per ogni variante di potenza):\n"
            "[\n"
            "  {\n"
            '    "Technology": "...",\n'
            '    "Bifacial": "...",\n'
            '    "STC": "...",\n'
            '    "PTC": "...",\n'
            '    "A_c": "...",\n'
            '    "Length": "...",\n'
            '    "Width": "...",\n'
            '    "N_s": "...",\n'
            '    "I_sc_ref": "...",\n'
            '    "V_oc_ref": "...",\n'
            '    "I_mp_ref": "...",\n'
            '    "V_mp_ref": "...",\n'
            '    "alpha_sc": "...",\n'
            '    "beta_oc": "...",\n'
            '    "T_NOCT": "...",\n'
            '    "a_ref": "...",\n'
            '    "I_L_ref": "...",\n'
            '    "I_o_ref": "...",\n'
            '    "R_s": "...",\n'
            '    "R_sh_ref": "...",\n'
            '    "Adjust": "...",\n'
            '    "gamma_r": "...",\n'
            '    "BIPV": "..."\n'
            "  }\n"
            "]\n\n"

            "Here are the definitions of each fields:\n"
            "-- Bifacial indicates whether the PV module is bifacial (1) or not (0). Check for 'Rear side power gain' table.\n"
            "-- STC stands for Standard Test Conditions power, corresponding to 'Maximum Power (Pmax/W)' at 1000 W/m²\n"
            "-- PTC stands for PVUSA Test Conditions power (1000 W/m², 20 °C ambient, 1 m/s wind)\n"
            "-- A_c represents the aperture area (m^2). Calculate as Length * Width.\n"
            "-- Length is the length in meters (m). Convert from mm if necessary.\n"
            "-- Width is the width in meters (m). Convert from mm if necessary.\n"
            "-- N_s is the number of cells (es. 'Cell Orientation')\n"
            "-- I_sc_ref is the Short Circuit Current (Isc/A) at STC\n"
            "-- V_oc_ref is the Open Circuit Voltage (Voc/V) at STC\n"
            "-- I_mp_ref is the current at maximum power point at STC (A)\n"
            "-- V_mp_ref is the Voltage at maximum power point at STC (V)\n"
            "-- alpha_sc is the Temperature coefficient of short-circuit current (A/°C)\n"
            "-- beta_oc is the Temperature coefficient of open-circuit voltage (V/°C)\n"
            "-- T_NOCT is the nominal operating cell temperature (NOCT)\n"
            "-- a_ref is the Modified diode ideality factor × number of cells × thermal voltage\n"
            "-- I_L_ref is the Light-generated current at reference conditions (A)\n"
            "-- I_o_ref is the Diode saturation current at reference conditions (A)\n"
            "-- R_s is the series resistance (Ω)\n"
            "-- R_sh_ref is the Shunt resistance at STC (Ω)\n"
            "-- Adjust is the Empirical adjustment factor\n"
            "-- gamma_r is the Temperature coefficient of Pmp (%/°C)\n"
            "-- BIPV indicates whether the PV module is building-integrated (Y) or not (N)\n\n"

            "This is the rule:\n"
            "- IMPORTANTE: Genera un oggetto JSON per ogni colonna di dati trovata nelle tabelle elettriche.\n"
            "- Se un parametro non è presente nel PDF, usa il valore: \"NOT_FOUND\".\n"
            "- Non aggiungere commenti.\n"
            "- Non aggiungere testo fuori dal JSON.\n"
        )
    }
]

In [5]:
messages_pv = [
    {
        "role": "user",
        "content": (
            "Analizza ESCLUSIVAMENTE il contenuto del PDF riportato di seguito. "
            "Non utilizzare conoscenze esterne.\n\n"
            "Datasheets has different models isnide table, and I want this informarion to be filled and extracted from this datasheet for all the models\n"
            "=== PDF TEXT START ===\n"
            + pdf_text +
            "\n=== PDF TEXT END ===\n\n"
            "Il documento contiene tabelle tecniche per diversi modelli (Power Class). "
            "Il documento può contenere una tabella tecnica con più modelli di pv (Power Class) oppure uno unico. "
            "Importante: Alcuni parametri sono comuni a tutti i modelli (celle unite), mentre altri (come la sezione 'Output') sono specifici per ogni colonna/modello.\n"
            "Estrai i parametri specificati per OGNI modello identificato."
            "Restituisci esclusivamente un array JSON di oggetti, dove ogni oggetto rappresenta un modello.\n"
            "Usa questa struttura:\n"
            "[\n"
            "  {\n"
            '    "ModelName": "...",\n'
            '    "Technology": "...",\n'
            '    "Bifacial": "...",\n'
            '    "STC": "...",\n'
            '    "PTC": "...",\n'
            '    "A_c": "...",\n'
            '    "Length": "...",\n'
            '    "Width": "...",\n'
            '    "N_s": "...",\n'
            '    "I_sc_ref": "...",\n'
            '    "V_oc_ref": "...",\n'
            '    "I_mp_ref": "...",\n'
            '    "V_mp_ref": "...",\n'
            '    "alpha_sc": "...",\n'
            '    "beta_oc": "...",\n'
            '    "T_NOCT": "...",\n'
            '    "a_ref": "...",\n'
            '    "I_L_ref": "...",\n'
            '    "I_o_ref": "...",\n'
            '    "R_s": "...",\n'
            '    "R_sh_ref": "...",\n'
            '    "Adjust": "...",\n'
            '    "gamma_r": "...",\n'
            '    "BIPV": "..."\n'
            "  }\n"
            "]\n\n"

            "Here are the definitions of each fields:\n"
            "-- Technology: Tipologia di cella (es. Q.ANTUM NEO solar half cells).\n"
            "-- Bifacial: Indica se il modulo è bifacciale (1) o no (0). Cerca note su 'Bifaciality' o 'rear side irradiation'.\n"
            "-- STC: Potenza massima al punto di massima potenza (Pmp o Pmax) in condizioni STC.\n"
            "-- PTC: Potenza in condizioni PTC. Se non presente, usa \"NOT_FOUND\".\n"
            "-- A_c: Area del modulo (m^2). Calcola come Length * Width.\n"
            "-- Length: Lunghezza in metri (m). Converti i mm (es. 1762 mm -> 1.762).\n"
            "-- Width: Larghezza in metri (m). Converti i mm (es. 1134 mm -> 1.134).\n"
            "-- N_s: Numero di celle in serie (es. 6 x 16 -> 96).\n"
            "-- I_sc_ref: Corrente di corto circuito (Isc) a STC.\n"
            "-- V_oc_ref: Tensione a circuito aperto (Voc) a STC.\n"
            "-- I_mp_ref: Corrente al punto di massima potenza (Impp) a STC.\n"
            "-- V_mp_ref: Tensione al punto di massima potenza (Vmpp) a STC.\n"
            "-- alpha_sc: Coefficiente di temperatura di Isc (%/K o A/°C).\n"
            "-- beta_oc: Coefficiente di temperatura di Voc (%/K o V/°C).\n"
            "-- T_NOCT: Temperatura nominale operativa della cella (NMOT o NOCT).\n"
            "-- a_ref, I_L_ref, I_o_ref, R_s, R_sh_ref, Adjust: Parametri del modello a 5 parametri. Se non presenti, usa \"NOT_FOUND\".\n"
            "-- gamma_r: Coefficiente di temperatura di Pmp (%/K).\n"
            "-- BIPV: Indica se integrato negli edifici (Y/N). Default 'N' se non specificato.\n\n"

            "This is the rule:\n"
            "- IMPORTANTE: Crea un oggetto JSON separato per ogni colonna di potenza.\n"
            "- Se un parametro non è presente nel PDF, usa il valore: \"NOT_FOUND\".\n"
            "- Non aggiungere commenti o testo fuori dal JSON.\n"
            "- Il JSON deve essere sintatticamente valido."
        )
    }
]

In [6]:
payload_completion = {
    "model": "falcon3:7b", 
    "stream": False,
    "params": {
        "temperature": 0.5,
        "top_p": 0.85
    },
    "messages": messages_pv,
    "files": [
        {
            "id": file_id,
            "type": "file"
        }
    ]
}
url_completion = f"{BASE_URL}/api/chat/completions"
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]
print(f"{answer}")

Based on the provided information and the structure you've outlined, here's how the data might be extracted and formatted into a JSON array:

```json
[
  {
    "ModelName": "TSM-425",
    "Technology": "N-type i-TOPCon Monocrystalline",
    "Bifacial": "1",
    "STC": "425 W",
    "PTC": "NOT_FOUND",
    "A_c": "1.762 * 1.134",
    "Length": "1.762 m",
    "Width": "1.134 m",
    "N_s": "144 cells",
    "I_sc_ref": "NOT_FOUND",
    "V_oc_ref": "NOT_FOUND",
    "I_mp_ref": "NOT_FOUND",
    "V_mp_ref": "NOT_FOUND",
    "alpha_sc": "NOT_FOUND",
    "beta_oc": "NOT_FOUND",
    "T_NOCT": "43 °C (±2 °C)",
    "a_ref": "NOT_FOUND",
    "I_L_ref": "NOT_FOUND",
    "I_o_ref": "NOT_FOUND",
    "R_s": "NOT_FOUND",
    "R_sh_ref": "NOT_FOUND",
    "Adjust": "NOT_FOUND",
    "gamma_r": "NOT_FOUND",
    "BIPV": "N"
  },
  {
    "ModelName": "TSM-430",
    "Technology": "N-type i-TOPCon Monocrystalline",
    "Bifacial": "1",
    "STC": "430 W",
    "PTC": "NOT_FOUND",
    "A_c": "1.762 * 1.134",
    

### DELETE CHAT AND PDF

In [16]:
import requests


url_completed = f"{BASE_URL}/api/chat/completed"
close_chat = requests.post(url_completed, headers=HEADERS_JSON)

response = requests.delete(f"{BASE_URL}/api/v1/chats/{chat_id}", headers=HEADERS_FILE)

if response.status_code not in (200, 204):
    raise Exception(f"Errore cancellazione chat: {response.status_code} - {response.text}")
print(f"Chat {chat_id} eliminata correttamente.")


Chat f70aa342-403c-4975-b571-f120436010ab eliminata correttamente.


In [15]:
import subprocess


file_path = file_data["path"]

result = subprocess.run(["docker", "exec", "open-webui", "rm", "-f", file_path], 
    capture_output=True, text=True)

if result.returncode == 0:
    print(f"File PDF {file_id} eliminato correttamente.")
else:
    print("Errore eliminazione file:", result.stderr)

File PDF 56b827a2-ae67-4d9c-a5de-bd1226fb0ff8 eliminato correttamente.
